# Overview of notebook

This notebook solves a few sample convex optimization problems using code I built from scratch. To validate the code, I compare against solutions computed by the `CVXPY` package.

I solve three types of problems:
1. Equality constrained quadratic problem
2. Equality constrained problem
3. Inequality and equality constrained problem

In [1]:
import numpy as np
import cvxpy

import convex_problem
import convex_functions

%load_ext autoreload
%autoreload 2

# Equality-constrained quadratic problem
Solve the problem
$$
\begin{equation*}
\begin{aligned}
&\text{minimize} & & \frac{1}{2} x^T P x + q^T x + r \\
&\text{subject to} & & Ax = b
\end{aligned}
\end{equation*}
$$

where the decision variable is $x \in \mathbb{R}^n$.

#### Generate problem data

In [2]:
n = 50
m = 40

U = np.random.randn(n, n)
P = U.T.dot(U)
q = np.random.randn(n)
r = 0

A = np.random.randn(m, n)
b = A.dot(np.random.randn(n))


#### Solve with CVXPY

In [3]:
x = cvxpy.Variable(n)
problem = cvxpy.Problem(
    objective=cvxpy.Minimize(0.5*cvxpy.quad_form(x, P) + q.T@x), 
    constraints=[A @ x == b]
)

solution_cvxpy = problem.solve()
x_cvxpy = x.value


#### Solve with my code

In [4]:
problem = convex_problem.QuadraticEqualityConstrained(P,q,r,A,b)
solution, x = problem.solve()


#### Validate solution

In [5]:
print('CVXPY solution: {:.5f}'.format(solution_cvxpy))
print('My solution:    {:.5f}'.format(solution))
print(f'My solution satisfies equality constraints:   {np.linalg.norm(A.dot(x) - b) < 1e-4}')
print('')
print(f'CVXPY optimal x: \n{x_cvxpy.round(4)}')
print(f'My optimal x: \n{x.round(4)}')
relative_error_norm = np.linalg.norm(x_cvxpy - x)/np.linalg.norm(x_cvxpy)
print('norm(my_optimal_x - cvxpy_optimal_x) / norm(cvxpy_optimal_x): {}'.format(relative_error_norm))


CVXPY solution: 1251.20272
My solution:    1251.20272
My solution satisfies equality constraints:   True

CVXPY optimal x: 
[-1.0292  2.0068 -0.0433 -0.4221 -0.2015 -0.5589 -1.2816 -0.3297  1.9903
  0.8329 -0.0156  0.5206 -1.615   2.4906  0.0255 -2.3752 -0.6318  0.014
  0.104  -1.6877  2.6405 -1.0976 -1.9651  0.4448  0.4493  1.3571  0.333
 -0.1354 -0.9154 -0.7802  0.932  -0.1445  2.3963  1.294  -1.5534  1.0105
 -0.7343  1.3388 -0.1624  0.4978  0.8753 -0.6347  0.3588 -0.7816 -0.4947
  1.0109  0.182   1.6875 -0.2262  0.9095]
My optimal x: 
[-1.0292  2.0068 -0.0433 -0.4221 -0.2015 -0.5589 -1.2816 -0.3297  1.9903
  0.8329 -0.0156  0.5206 -1.615   2.4906  0.0255 -2.3752 -0.6318  0.014
  0.104  -1.6877  2.6405 -1.0976 -1.9651  0.4448  0.4493  1.3571  0.333
 -0.1354 -0.9154 -0.7802  0.932  -0.1445  2.3963  1.294  -1.5534  1.0105
 -0.7343  1.3388 -0.1624  0.4978  0.8753 -0.6347  0.3588 -0.7816 -0.4947
  1.0109  0.182   1.6875 -0.2262  0.9095]
norm(my_optimal_x - cvxpy_optimal_x) / norm(cvxpy_o

# Equality-constrained problem
Solve the problem
$$
\begin{equation*}
\begin{aligned}
&\text{minimize} & & || x ||_2^2 + \sum_{i=1}^n \exp(x_i) \\
&\text{subject to} & & Ax = b
\end{aligned}
\end{equation*}
$$
where the decision variable is $x \in \mathbb{R}^n$.

#### Generate problem data

In [6]:
n = 40
m = 80

A = np.random.randn(m, n)
b = A.dot(np.random.randn(n))


class MyFunction:
    def __call__(self, x):
        return x.dot(x) + np.exp(x).sum()
    
    def gradient(self, x):
        return 2*x + np.exp(x)
    
    def hessian(self, x):
        return 2*np.eye(x.shape[0]) + np.diag(np.exp(x))

#### Solve with CVXPY

In [7]:
x = cvxpy.Variable(n)
problem = cvxpy.Problem(
    objective=cvxpy.Minimize(cvxpy.norm(x)**2 + cvxpy.sum(cvxpy.exp(x))), 
    constraints=[A @ x == b]
)
solution_cvxpy = problem.solve()
x_cvxpy = x.value


#### Solve with my code

In [8]:
starting_point = np.linalg.lstsq(A, b, rcond=None)[0]
function = MyFunction()

problem = convex_problem.EqualityConstrained(function, A, b)
solution, x = problem.solve(starting_point)


#### Validate solution

In [9]:
print('CVXPY solution: {:.5f}'.format(solution_cvxpy))
print('My solution:    {:.5f}'.format(solution))
print(f'My solution satisfies equality constraints:   {np.linalg.norm(A.dot(x) - b) < 1e-4}')
print('')
print(f'CVXPY optimal x: \n{x_cvxpy.round(4)}')
print(f'My optimal x: \n{x.round(4)}')
relative_error_norm = np.linalg.norm(x_cvxpy - x)/np.linalg.norm(x_cvxpy)
print('norm(my_optimal_x - cvxpy_optimal_x) / norm(cvxpy_optimal_x): {}'.format(relative_error_norm))


CVXPY solution: 110.85618
My solution:    110.85619
My solution satisfies equality constraints:   True

CVXPY optimal x: 
[-1.5231  0.3271 -1.2208  0.9992  1.6103  1.222   0.358  -0.8122 -0.4741
  1.0324  1.546  -0.0987 -0.379   0.5573 -0.1444  0.1843  0.6053 -0.2543
 -0.2369  0.9516 -0.2372  0.2495  0.7851  0.1135 -0.0598 -0.083   1.8809
  0.8666  0.4163  2.1166  1.4971 -0.5367 -1.7822  0.5987  0.0279 -0.0474
 -1.6664  0.0938  0.4111  1.1583]
My optimal x: 
[-1.5231  0.3271 -1.2208  0.9992  1.6103  1.222   0.358  -0.8122 -0.4741
  1.0324  1.546  -0.0987 -0.379   0.5573 -0.1444  0.1843  0.6053 -0.2543
 -0.2369  0.9516 -0.2372  0.2495  0.7851  0.1135 -0.0598 -0.083   1.8809
  0.8666  0.4163  2.1166  1.4971 -0.5367 -1.7822  0.5987  0.0279 -0.0474
 -1.6664  0.0938  0.4111  1.1583]
norm(my_optimal_x - cvxpy_optimal_x) / norm(cvxpy_optimal_x): 3.1070220761400647e-09


# Equality and inequality constrained problem
Solve the problem
$$
\begin{equation*}
\begin{aligned}
&\text{minimize} & & c^T x \\
&\text{subject to} & & x^T x - 1 \leq 0 \\
& & & Ax = b \\
\end{aligned}
\end{equation*}
$$
where the decision variable is $x \in \mathbb{R}^n$.

#### Generate problem data

In [10]:
n = 50
m = 100

c = np.random.randint(-50,50,n)
A = np.random.randn(m, n)
x_0 = np.random.uniform(size=n) / np.sqrt(n)
b = A.dot(x_0)


#### Solve with CVXPY

In [11]:
x = cvxpy.Variable(n)
problem = cvxpy.Problem(
    objective=cvxpy.Minimize(c.T @ x), 
    constraints=[cvxpy.norm(x)**2 <= 1, A @ x == b]
)

solution_cvxpy = problem.solve()
x_cvxpy = x.value


#### Solve with my code

In [12]:
class MyObjectiveFunction:
    '''Represents the function f(x) = c^T x'''
    def __init__(self, c):
        self.c = c
        self.n = c.shape[0]
    
    def __call__(self, x):
        return self.c.dot(x)
    
    def gradient(self, x):
        return self.c
    
    def hessian(self, x):
        return np.zeros((self.n, self.n))
    
class MyConstraintFunction:
    '''Represents the function f(x) = x^T x - 1'''
    def __call__(self, x):
        return x.dot(x) - 1
    
    def gradient(self, x):
        return 2*x
    
    def hessian(self, x):
        return 2*np.eye(x.shape[0])


objective = MyObjectiveFunction(c)
constraint_functions = [MyConstraintFunction()]

problem = convex_problem.EqualityAndInequalityConstrained(objective, constraint_functions, A, b)
solution, x = problem.solve(x_0)


#### Validate solution

In [13]:
print('CVXPY solution: {:.5f}'.format(solution_cvxpy))
print('My solution:    {:.5f}'.format(solution))
print(f'My solution satisfies equality constraints:   {np.linalg.norm(A.dot(x) - b) < 1e-4}')
print(f'My solution satisfies inequality constraints: {all([f(x) < 0 for f in constraint_functions])}')
print('')
print(f'CVXPY optimal x: \n{x_cvxpy.round(4)}')
print(f'My optimal x: \n{x.round(4)}')
relative_error_norm = np.linalg.norm(x_cvxpy - x)/np.linalg.norm(x_cvxpy)
print('norm(my_optimal_x - cvxpy_optimal_x) / norm(cvxpy_optimal_x): {}'.format(relative_error_norm))


CVXPY solution: -20.41636
My solution:    -20.41629
My solution satisfies equality constraints:   True
My solution satisfies inequality constraints: True

CVXPY optimal x: 
[0.0352 0.0458 0.0048 0.0253 0.0082 0.0014 0.019  0.1088 0.1237 0.1091
 0.1126 0.1116 0.1185 0.0857 0.0871 0.1107 0.0434 0.0432 0.0256 0.0531
 0.0531 0.0093 0.0593 0.0348 0.0761 0.0237 0.0941 0.1057 0.1266 0.0629
 0.1048 0.0345 0.1123 0.0987 0.1201 0.0245 0.0551 0.0513 0.0318 0.0386
 0.0528 0.0739 0.0202 0.1052 0.0263 0.0757 0.0777 0.103  0.0386 0.095 ]
My optimal x: 
[0.0352 0.0458 0.0048 0.0253 0.0082 0.0014 0.019  0.1088 0.1237 0.1091
 0.1126 0.1116 0.1185 0.0857 0.0871 0.1107 0.0434 0.0432 0.0256 0.0531
 0.0531 0.0093 0.0593 0.0348 0.0761 0.0237 0.0941 0.1057 0.1266 0.0629
 0.1048 0.0345 0.1123 0.0987 0.1201 0.0245 0.0551 0.0513 0.0318 0.0386
 0.0528 0.0739 0.0202 0.1052 0.0263 0.0757 0.0777 0.103  0.0386 0.095 ]
norm(my_optimal_x - cvxpy_optimal_x) / norm(cvxpy_optimal_x): 2.971953928332515e-06
